In [2]:
%run -i 'create_tables.py'

Connecting to redshift
Connected to redshift
Dropping existing tables if any
Executing drop: DROP TABLE IF EXISTS staging_events
Executing drop: DROP TABLE IF EXISTS staging_songs
Executing drop: DROP TABLE IF EXISTS songplays
Executing drop: DROP TABLE IF EXISTS users
Executing drop: DROP TABLE IF EXISTS songs
Executing drop: DROP TABLE IF EXISTS artists
Executing drop: DROP TABLE IF EXISTS time
Creating tables
Executing create: CREATE TABLE staging_events(
    event_id INT IDENTITY(0,1),
    artist_name VARCHAR(255),
    auth VARCHAR(50),
    user_first_name VARCHAR(255),
    user_gender  VARCHAR(1),
    item_in_session	INTEGER,
    user_last_name VARCHAR(255),
    song_length	DOUBLE PRECISION, 
    user_level VARCHAR(50),
    location VARCHAR(255),	
    method VARCHAR(25),
    page VARCHAR(35),	
    registration VARCHAR(50),	
    session_id	BIGINT,
    song_title VARCHAR(255),
    status INTEGER,  -- SHOULD BE ALWAYS A NUMBER, HTTP STATUS	
    ts VARCHAR(50),
    user_agent TEXT,	
 

In [3]:
%run -i 'etl.py'

Connecting to redshift
Connected to redshift
Loading staging tables
Transform from staging
Transform data by: INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) 
    SELECT DISTINCT 
        TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second' as start_time, 
        e.user_id, 
        e.user_level,
        s.song_id,
        s.artist_id,
        e.session_id,
        e.location,
        e.user_agent
    FROM staging_events e, staging_songs s
    WHERE e.page = 'NextSong'
    AND e.song_title = s.title
    AND user_id NOT IN (SELECT DISTINCT s.user_id FROM songplays s WHERE s.user_id = user_id
                       AND s.start_time = start_time AND s.session_id = session_id )

Transform data by: INSERT INTO users (user_id, first_name, last_name, gender, level)  
    SELECT DISTINCT 
        user_id,
        user_first_name,
        user_last_name,
        user_gender, 
        user_level
    FROM staging_events
    WHERE page = 'NextS